<a href="https://colab.research.google.com/github/rmham93/Master_thesis/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision 
import torch 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
!pip install kaggle

In [3]:
from google.colab import files #Kaggle API key
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rmham93","key":"e74af0eb2d4f3915e356c8fc28bae8e1"}'}

In [4]:
!mkdir -p ~/ .kaggle
!cp -r kaggle.json ~/ .kaggle/
!chmod 600 ~/ .kaggle/kaggle.json

In [5]:
!mv .kaggle /root/

In [6]:
!kaggle datasets download -d rahulbhalley/fashion-usage

100% 5.35G/5.36G [01:47<00:00, 49.6MB/s]
100% 5.36G/5.36G [01:47<00:00, 53.7MB/s]


In [7]:
!ls

fashion-usage.zip  kaggle.json	sample_data


In [ ]:
!unzip fashion-usage.zip

In [9]:
!ls

fashion-usage.zip  kaggle.json	sample_data  usage


In [10]:
!cd /content/usage/
!pwd

/content


In [11]:
!mkdir dataset 
!mv /content/usage/Casual dataset/Casual
!mv /content/usage/Ethnic dataset/Ethnic
!mv /content/usage/Formal dataset/Formal
!mv /content/usage/Party dataset/Party
!mv /content/usage/Smart\ Casual dataset/SmartCasual
!mv /content/usage/Sports dataset/Sports
!mv /content/usage/Travel dataset/Travel

In [12]:
from PIL import Image
import glob
import numpy as np
import matplotlib.pyplot as plt

In [13]:
images= []

casual_list = []
for filename in glob.glob('dataset/Casual/*.jpg'): #assuming gif
    im=Image.open(filename).convert('LA')
    im = im.resize((50,50))
    casual_list.append(np.array(im))
    #plt.imshow(casual_list[0])

#print(len(casual_list[0]))
images.append(np.array(casual_list))

print("passed 1")

ethnic_list = []
for filename in glob.glob('dataset/Ethnic/*.jpg'): #assuming gif
    im=Image.open(filename).convert('LA')
    im = im.resize((50,50))
    ethnic_list.append(np.array(im))
#print("Ethnic ", len(ethnic_list))

images.append(np.array(ethnic_list))

print("passed 2")

formal_list = []
for filename in glob.glob('dataset/Formal/*.jpg'): #assuming gif
    im=Image.open(filename).convert('LA')
    im = im.resize((50,50))
    formal_list.append(np.array(im))
#print("Ethnic ", len(ethnic_list))

images.append(np.array(formal_list))

print("passed 3")

party_list = []
for filename in glob.glob('dataset/Party/*.jpg'): #assuming gif
    im=Image.open(filename).convert('LA')
    im = im.resize((50,50))
    party_list.append(np.array(im))
#print("Ethnic ", len(ethnic_list))

images.append(np.array(party_list))

print("passed 4")

smartcasual_list =[]
for filename in glob.glob('dataset/SmartCasual/*.jpg'): #assuming gif
    im = Image.open( filename ).convert('LA')
    im = im.resize((50,50))
    smartcasual_list.append(np.array(im))
#print("Ethnic ", len(ethnic_list))

images.append(np.array(smartcasual_list))

print("passed 5")

sport_list =[]
for filename in glob.glob('dataset/Sport/*.jpg'): #assuming gif
    im = Image.open( filename ).convert('LA')
    im = im.resize((50,50))
    sport_list.append(np.array(im))
#print("Ethnic ", len(ethnic_list))

images.append(np.array(sport_list))

print("passed 6")

travel_list =[]
for filename in glob.glob('dataset/Travel/*.jpg'): #assuming gif
    im = Image.open( filename ).convert('LA')
    im = im.resize((50,50))
    travel_list.append(np.array(im))
#print("Ethnic ", len(ethnic_list))

images.append(np.array(travel_list))

print("passed 7")

np.save('full_data.npy', images) # save
#new_num_arr = np.load('data.npy') # load

passed 1
passed 2
passed 3
passed 4
passed 5
passed 6
passed 7


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!mv "/content/full_data.npy" "/content/drive/MyDrive/Colab Notebooks/"

### Result 
<p>
As a result, in this code I took data from kaggle, I have changed to grayscale and resized images since they were huge.
<p> I saved images into numpy array, and saved it as full_data.npy. Right now data is ready to be used from google drive. 
The rest wont be used here. I will fix those and use resnet in another code file

In [ ]:
model = models.resnet50(pretrained=True) #resneti indirelim

for isim, param in model.named_parameters():
    
    if 'layer4' not in isim:
      param.requires_grad = False

model.fc = torch.nn.Linear(2048, 7)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

In [ ]:
dataset = datasets.ImageFolder('./dataset', transform=train_transforms)
dataset_size= len(dataset)
train_size = int(dataset_size * 0.75)
test_boyutu = dataset_size - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_boyutu])
print('Dataset boyutu: ', len(dataset))

Dataset boyutu:  44123


In [ ]:
train_dloader = torch.utils.data.DataLoader(train_set,
                                          batch_size=128,
                                          shuffle=True)

test_dloader = torch.utils.data.DataLoader(test_set,
                                          batch_size=64,
                                          shuffle=False)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
total_epoch = 20
model = model.cuda().train()
for epoch in range(total_epoch):
    running_loss = 0
    for X, y in train_dloader:

      optimizer.zero_grad()

      X = X.cuda()
      y = y.cuda()

      cikti = model(X)

      #y = y.unsqueeze(1).long()

      loss = criterion(cikti, y)

      loss.backward()

      optimizer.step()

      running_loss += loss.data
      
    dogru = 0
    toplam = 0
    # test prosedürü 
    with torch.no_grad():
        for X, y in test_dloader:
          X = X.cuda() 
          y = y.cuda()
          
          cikti = model(X)
          
          _, maksimum = torch.max(cikti.data, 1)
          
          toplam += y.size(0)
          
          dogru += (maksimum == y).sum().item()

    print(epoch, running_loss, dogru/toplam)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
!wget https://assets.vogue.com/photos/6053b5288d30cc7b937c6506/1:1/w_1013,h_1013,c_limit/slide_17.jpg

In [ ]:
from PIL import Image
from IPython.display import display

image = Image.open('slide_17.jpg')

display(image)

image = train_transforms(image)

image = image.unsqueeze(0).cuda()


In [ ]:
model = model.eval()
with torch.no_grad():
  cikti_test = model(image)

In [ ]:
label = ['Casual', 'Ethnic', 'Formal','Party','Sports','Travel','Smart Casual']

_, max = torch.max(cikti_test.data, 1)

print("Prediction: ***", label[max.item()], "***")